In [38]:
!pip install scikit-learn

In [39]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import BayesianRidge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

In [40]:
%pip install openpyxl

# --------------------------
# 1. Carregar e preparar os dados
# --------------------------
df_biscobis = pd.read_excel('/Users/fabicampanari/Desktop/class_12- Bayesian-KNN Regression-Model Persistence/class__13-Exam-Bayesian-KNN Regression-Model Persistence/Biscobis.xlsx')
df_biscobis

Note: you may need to restart the kernel to use updated packages.


,x1,x2,x3,x4,x5,x6,x7,x8,x9
0,4.1,0.6,6.9,4.7,2.4,2.3,5.2,1,32
1,6.0,0.9,9.6,7.8,3.4,4.6,4.5,3,58
2,4.6,2.4,9.5,6.6,3.5,4.5,7.6,1,46
3,5.5,1.6,9.4,4.7,3.5,3.0,7.6,3,63
4,2.4,1.6,8.8,4.8,2.0,2.8,5.8,1,32
...,...,...,...,...,...,...,...,...,...
95,2.9,1.2,7.3,6.1,2.0,2.5,8.0,1,34
96,1.9,2.7,5.0,4.9,2.2,2.5,8.2,1,36
97,0.6,1.6,6.4,5.0,0.7,2.1,8.4,1,25
98,2.0,2.8,5.2,5.0,2.4,2.7,8.4,1,38


In [41]:
X_df_biscobis = df_biscobis[['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7']]
y_df_biscobis = df_biscobis['x9']

# Normalização
scaler = StandardScaler()
X_scaled_X_df_biscobis = scaler.fit_transform(X_df_biscobis)

In [42]:
X_df_biscobis

,x1,x2,x3,x4,x5,x6,x7
0,4.1,0.6,6.9,4.7,2.4,2.3,5.2
1,6.0,0.9,9.6,7.8,3.4,4.6,4.5
2,4.6,2.4,9.5,6.6,3.5,4.5,7.6
3,5.5,1.6,9.4,4.7,3.5,3.0,7.6
4,2.4,1.6,8.8,4.8,2.0,2.8,5.8
...,...,...,...,...,...,...,...
95,2.9,1.2,7.3,6.1,2.0,2.5,8.0
96,1.9,2.7,5.0,4.9,2.2,2.5,8.2
97,0.6,1.6,6.4,5.0,0.7,2.1,8.4
98,2.0,2.8,5.2,5.0,2.4,2.7,8.4


In [43]:
# Adicionar intercepto para GD e SGD
X_scaled_intercept = np.hstack((np.ones((X_scaled_X_df_biscobis.shape[0], 1)), X_scaled_X_df_biscobis))

# Divisão treino/teste
X_train, X_test, y_train, y_test = train_test_split(X_scaled_intercept, y_df_biscobis, test_size=0.2, random_state=42)
X_train_knn, _, y_train_knn, _ = train_test_split(X_scaled_X_df_biscobis, y_df_biscobis, test_size=0.2, random_state=42)  # KNN usa sem intercepto


In [44]:
# --------------------------
# 2. Regressão Bayesiana
# --------------------------
bayes_model = BayesianRidge()
bayes_model.fit(X_train[:, 1:], y_train)  # Remove intercepto para modelo scikit
y_pred_bayes = bayes_model.predict(X_train[:, 1:])
mse_bayes = mean_squared_error(y_train, y_pred_bayes)


In [45]:
# --------------------------
# 3. KNN
# --------------------------
knn_model = KNeighborsRegressor(n_neighbors=5)
knn_model.fit(X_train_knn, y_train_knn)
y_pred_knn = knn_model.predict(X_train_knn)
mse_knn = mean_squared_error(y_train_knn, y_pred_knn)

In [46]:
# --------------------------
# 4. Gradiente Descendente
# --------------------------
def gradient_descent(X, y, lr=0.01, n_iter=1000):
    m, n = X.shape
    theta = np.zeros(n)
    for _ in range(n_iter):
        gradient = -2/m * X.T.dot(y - X.dot(theta))
        theta -= lr * gradient
    return theta

theta_gd = gradient_descent(X_train, y_train.values, lr=0.01, n_iter=1000)
y_pred_gd = X_train.dot(theta_gd)
mse_gd = mean_squared_error(y_train, y_pred_gd)


In [47]:
# --------------------------
# 5. Gradiente Descendente Estocástico
# --------------------------
def stochastic_gradient_descent(X, y, lr=0.01, n_iter=1000):
    m, n = X.shape
    theta = np.zeros(n)
    for _ in range(n_iter):
        for i in range(m):
            idx = np.random.randint(m)
            xi = X[idx, :].reshape(1, -1)
            yi = y[idx]
            gradient = -2 * xi.T.dot(yi - xi.dot(theta))
            theta -= lr * gradient.ravel()
    return theta

theta_sgd = stochastic_gradient_descent(X_train, y_train.values, lr=0.01, n_iter=1000)
y_pred_sgd = X_train.dot(theta_sgd)
mse_sgd = mean_squared_error(y_train, y_pred_sgd)

In [48]:
# --------------------------
# 6. Resultados Finais
# --------------------------
resultados = {
    "Regressão Bayesiana": mse_bayes,
    "KNN": mse_knn,
    "Gradiente Descendente": mse_gd,
    "Gradiente Descendente Estocástico": mse_sgd
}

melhor_modelo = min(resultados, key=resultados.get)
melhor_mse = resultados[melhor_modelo]

print("🇧🇷Resultados (Erro Quadrático Médio - MSE):")
for modelo, mse in resultados.items():
    destaque = "🔥 MELHOR" if modelo == melhor_modelo else ""
    print(f"➡ {modelo}: {round(mse, 4)} {destaque}")

print(f"\n🚀 Melhor modelo com base na base de treino: **{melhor_modelo}** (MSE = {round(melhor_mse, 4)})")


🇧🇷Resultados (Erro Quadrático Médio - MSE):
➡ Regressão Bayesiana: 17.5955 
➡ KNN: 21.497 
➡ Gradiente Descendente: 17.5372 🔥 MELHOR
➡ Gradiente Descendente Estocástico: 17.9002 

🚀 Melhor modelo com base na base de treino: **Gradiente Descendente** (MSE = 17.5372)


🛸๋*ੈ✩* 🔭✮☾𖤓.☘︎ ݁˖⁷⁷⁷ㅤ✮ ⋆ ˚｡𖦹 ⋆｡°✩ ✮ ⋆ ˚｡𖦹 ⋆｡°✩  *ੈ✩‧₊🛸๋*ੈ✩* 🔭✮☾𖤓.☘︎ ݁˖⁷⁷⁷ㅤ✮ ⋆ ˚｡𖦹 ⋆｡°✩ ✮ ⋆ ˚｡𖦹 ⋆｡°✩  *ੈ✩‧₊

#### 🇺🇸Results (Mean Squared Error - MSE):
➡ Bayesian Regression: 17.5955 
➡ KNN: 21.497 
➡ Gradient Descent: 17.5372 🔥 BEST
➡ Stochastic Gradient Descent: 18.2634 

🚀 Best model based on the training set: **Gradient Descent** (MSE = 17.5372)